In [23]:
%run webscraper.ipynb

Back to Schedule
Geelong Cats
vs
North Melbourne Kangaroos
Live Game
Game
Money Line
Geelong Cats
North Melbourne Kangaroos
+ 2
Spread
Geelong Cats
-32.5
1.84
North Melbourne Kangaroos
+32.5
1.84
+ 2
Total
Over
148.5
1.84
Under
148.5
1.84
Geelong Cats
North Melbourne Kangaroos
Live Game
Game
Geelong Cats
North Melbourne Kangaroos
Geelong Cats
-32.5 1.84
North Melbourne Kangaroos
+32.5 1.84
Over
148.5 1.84
Under
148.5 1.84
            team 1              team 2 status date
0  Aguada Santeros  Criollos de Caguas   live     
            team 1              team 2 status date
0  Aguada Santeros  Criollos de Caguas   live     
           team 1         team 2   status         date
0  Atlanta Braves  New York Mets  pregame  07/26 16:10
           team 1         team 2   status         date
0  Atlanta Braves  New York Mets  pregame  07/26 16:10
            team 1          team 2 status date
0  Minnesota Twins  Detroit Tigers   live     
                          team 1              team 2   s

,Event Name,Outcome,Value,Odds
0,Game,Geelong Cats,-32.5,1.84
1,Game,North Melbourne Kangaroos,+32.5,1.84
2,Game,Over,148.5,1.84
3,Game,Under,148.5,1.84


Back to Schedule
Geelong Cats
vs
North Melbourne Kangaroos
Live Game
Game
Money Line
Geelong Cats
North Melbourne Kangaroos
+ 2
Spread
Geelong Cats
-32.5
1.84
North Melbourne Kangaroos
+32.5
1.84
+ 2
Total
Over
148.5
1.84
Under
148.5
1.84
  Event Name                    Outcome  Value  Odds
0       Game               Geelong Cats  -32.5  1.84
1       Game  North Melbourne Kangaroos  +32.5  1.84
2       Game                       Over  148.5  1.84
3       Game                      Under  148.5  1.84
Parsed DataFrame from extracted_text1:
Empty DataFrame
Columns: [Bet Type, Team/Outcome, Value, Odds]
Index: []

Parsed DataFrame from extracted_text2:
Empty DataFrame
Columns: [Bet Type, Team/Outcome, Value, Odds]
Index: []
     Bet Type Team/Outcome Value  Odds
0      Spread    ATHLETICS  -1.5  -110
1      Spread       ANGELS  +1.5  -120
2       Total         OVER  14.5  -140
3       Total        UNDER  14.5  +110
4  Money Line    ATHLETICS  None  -225
5  Money Line       ANGELS  None  +17

In [24]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
import pandas as pd

In [25]:
def open_nowgoal(driver, url):
    driver.get(url)
    wait = WebDriverWait(driver, 10)
    
    try:
        # Locate and click the "Trends" button
        trends_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'team-table-other')))
        trends_button.click()
        time.sleep(1)  # Wait for data to load after clicking trends button

    except Exception as e:
        print(f"Failed to expand data or click trends button: {e}")
    
    return driver.page_source

In [26]:
# Function to parse the data
def parse_raw_data(data):
    # Replace all tabs with spaces for uniformity
    data = data.replace('\t', ' ')

    # Split the entire data into individual words
    words = data.split()

    # Prepare lists to store each column's data
    periods = []
    quarters = []
    clocks = []
    scores = []
    home_odds = []
    handicaps = []
    away_odds = []
    statuses = []
    days = []
    updates = []

    # Regular expression to match quarters
    quarter_pattern = re.compile(r"(1st|2nd|3rd|4th)")
    time_pattern = re.compile(r"^\d{2}:\d{2}$")  # Regex for time format xx:xx

    # Iterate over each word
    i = 0
    while i < len(words):
        # Check for a quarter start
        if quarter_pattern.match(words[i]) and "Qtr" in words[i + 1]:
            # Extract the period
            period = words[i]
            quarter = words[i + 1]

            # Check if the next string is a valid time format
            i += 2
            if time_pattern.match(words[i]):
                # Valid time, proceed with extraction
                clock = words[i]
                clocks.append(clock)

                # Extract score
                i += 1
                score = words[i]
                scores.append(score)

                # Check if "Closed"
                i += 1
                if words[i] == "Closed":
                    home_odds.append("")
                    handicaps.append("")
                    away_odds.append("")
                    statuses.append("Closed")
                else:
                    # Extract home odds
                    home_odds.append(words[i])

                    # Extract handicap
                    i += 1
                    handicaps.append(words[i])

                    # Extract away odds
                    i += 1
                    away_odds.append(words[i])

                    # Set status as "Open"
                    statuses.append("Open")

                # Extract day and update time
                i += 1
                day = words[i]
                update_time = words[i + 1]

                # Store day and update time
                days.append(day)
                updates.append(update_time)

                # Store period and quarter after ensuring time is valid
                periods.append(period)
                quarters.append(quarter)

                # Move index past update time
                i += 2

            else:
                # If not valid time, skip to the next quarter
                print(f"Invalid time format detected: {words[i]}. Skipping to next quarter.")
                while i < len(words) and not quarter_pattern.match(words[i]):
                    i += 1

        else:
            # Increment index if not a quarter start
            i += 1

    # Create DataFrame
    df = pd.DataFrame({
        "Period": periods,
        "Quarter": quarters,
        "Clock": clocks,
        "Score": scores,
        "Home Odds": home_odds,
        "Handicap": handicaps,
        "Away Odds": away_odds,
        "Status": statuses,
        "Day": days,
        "Update": updates
    })

    # Switch the sign of Handicap or set to empty if there is no value
    df['Handicap'] = df['Handicap'].apply(lambda x: "" if x == "" else -float(x) if float(x) < 50 else float(x) )

    return df

# Function to calculate implied probabilities from odds
def calculate_implied_probabilities(df):
    # Convert odds columns to numeric, handling non-numeric values gracefully
    df['Home Odds'] = pd.to_numeric(df['Home Odds'], errors='coerce')
    df['Away Odds'] = pd.to_numeric(df['Away Odds'], errors='coerce')

    # Calculate implied probabilities
    df['Implied Probability Home'] = 1 / df['Home Odds']
    df['Implied Probability Away'] = 1 / df['Away Odds']

    # Handle closed odds by setting probabilities to NaN
    df.loc[df['Status'] == 'Closed', ['Implied Probability Home', 'Implied Probability Away']] = None

    return df


In [27]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def grabmatchinfo(driver):
    wait = WebDriverWait(driver, 10)  # 10 seconds timeout

    # Wait for the home team element to be present
    home_team_div = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div.home"))
    )

    # Extract home team information
    home_team_name = home_team_div.find_element(By.CSS_SELECTOR, "div.sclassName").text
    home_team_url = home_team_div.get_attribute("data-url")
    home_team_img = home_team_div.find_element(By.TAG_NAME, "img").get_attribute("src")
    print(f"Home Team: {home_team_name}, URL: {home_team_url}, Image: {home_team_img}")

    # Wait for the away team element to be present
    away_team_div = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div.guest"))
    )

    # Extract away team information
    away_team_name = away_team_div.find_element(By.CSS_SELECTOR, "div.sclassName").text
    away_team_url = away_team_div.get_attribute("data-url")
    away_team_img = away_team_div.find_element(By.TAG_NAME, "img").get_attribute("src")
    print(f"Away Team: {away_team_name}, URL: {away_team_url}, Image: {away_team_img}")

    # Wait for the "Sport" element to be present
    sport_element = wait.until(
        EC.presence_of_element_located((By.CLASS_NAME, "sclassLink"))
    )

    # Extract the Sport name
    sport_name = sport_element.text
    print(f"Sport: {sport_name}")

    # Wait for the "Match Date" element to be present
    match_date_element = wait.until(
        EC.presence_of_element_located((By.CLASS_NAME, "time"))
    )

    # Extract the Match Date from the data-t attribute
    match_date = match_date_element.get_attribute("data-t")
    print(f"Match Date: {match_date}")

    # Extracting additional game information from the 'vs' section
    score_div = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div#mScore .end"))
    )

    # Extract home and away scores
    home_score = score_div.find_element(By.CSS_SELECTOR, "div.score:nth-child(1)").text
    away_score = score_div.find_element(By.CSS_SELECTOR, "div.score:nth-child(3)").text

    # Extract game state (finished, ongoing, etc.)
    game_state = score_div.find_element(By.CSS_SELECTOR, "div.row.state b").text

    # Extract quarter scores
    quarter_scores = score_div.find_element(By.CSS_SELECTOR, "span.hscore").text

    print(f"Home Score: {home_score}")
    print(f"Away Score: {away_score}")
    print(f"Game State: {game_state}")
    print(f"Quarter Scores: {quarter_scores}")

    # Return a dictionary with all the extracted information
    match_info = {
        "home_team": {
            "name": home_team_name,
            "url": home_team_url,
            "image": home_team_img,
            "score": home_score
        },
        "away_team": {
            "name": away_team_name,
            "url": away_team_url,
            "image": away_team_img,
            "score": away_score
        },
        "sport": sport_name,
        "match_date": match_date,
        "game_state": game_state,
        "quarter_scores": quarter_scores
    }

    return match_info



In [28]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

def grabpregame(driver):
    # Wait for the table row to be present
    wait = WebDriverWait(driver, 10)  # 10 seconds timeout
    table_row = wait.until(
        EC.presence_of_element_located((By.XPATH, "//tr[@name='oddsTr' and @cid='8']"))
    )

    # Extract data from each cell in the row
    cells = table_row.find_elements(By.TAG_NAME, "td")
    extracted_data = {
        "Bookmaker": cells[0].text,
        "Status": [span.text for span in cells[1].find_elements(By.TAG_NAME, "span")],
        "Odds 1": [span.get_attribute("data-o") for span in cells[2].find_elements(By.TAG_NAME, "span")],
        "Handicap 1": [span.get_attribute("data-o") for span in cells[3].find_elements(By.TAG_NAME, "span")],
        "Odds 2": [span.get_attribute("data-o") for span in cells[4].find_elements(By.TAG_NAME, "span")],
        "Odds 3": [span.get_attribute("data-o") for span in cells[5].find_elements(By.TAG_NAME, "span")],
        "Odds 4": [span.get_attribute("data-o") for span in cells[7].find_elements(By.TAG_NAME, "span")],
        "Odds 5": [span.get_attribute("data-o") for span in cells[8].find_elements(By.TAG_NAME, "span")],
        "Total": [span.get_attribute("data-o") for span in cells[9].find_elements(By.TAG_NAME, "span")],
        "Odds 6": [span.get_attribute("data-o") for span in cells[10].find_elements(By.TAG_NAME, "span")],
        "Trends": cells[11].find_element(By.TAG_NAME, "a").get_attribute("onclick")
    }

    # Print the extracted data for debugging purposes
    print("Extracted Data:", extracted_data)
    
    # Optional: Convert to DataFrame for easier handling
    pregame_df = pd.DataFrame([extracted_data])
    print(pregame_df)

    # Return the extracted data as a dictionary
    return extracted_data


In [29]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time

def grablivegame(driver):
    # Create a WebDriverWait object with a timeout of 10 seconds
    wait = WebDriverWait(driver, 10)

    try:
        # Locate and click the "Trends" button
        button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, 'odd_a')))
        button.click()
        time.sleep(2)  # Wait for data to load after clicking the trends button

        # Grab all the text on the page for Q4 handicap odds
        handicap_odds_q4 = driver.page_source

        # Click on the button to change the odds type to "3" (Q4 total odds)
        button = wait.until(
            EC.element_to_be_clickable((By.XPATH, "//*[@onclick=\"_oddsDetailWin.checkedOddsType(3)\"]"))
        )
        button.click()
        time.sleep(2)  # Wait for data to load after clicking the button

        # Grab all the text on the page for Q4 total odds
        total_odds_q4 = driver.page_source

        # Hover over the element to reveal options
        hover_element = wait.until(
            EC.presence_of_element_located((By.ID, "preiod"))
        )

        # Perform the mouse hover action
        action = ActionChains(driver)
        action.move_to_element(hover_element).perform()

        # Wait for the dropdown or hidden elements to appear
        # Wait for the specific span element to be clickable (Q2 total odds)
        option_to_click = wait.until(
            EC.element_to_be_clickable((By.XPATH, "//span[@type='3']"))
        )
        # Click the desired option
        option_to_click.click()
        time.sleep(2)  # Optional: Wait for any potential data load

        # Grab all the text on the page for Q2 total odds
        total_odds_q2 = driver.page_source

        # Wait for the element with specific class and oddstype to be clickable
        wait = WebDriverWait(driver, 10)  # 10 seconds timeout
        ah_odds_button = wait.until(
            EC.element_to_be_clickable((By.XPATH, "//a[@class='' and @oddstype='1' and @onclick='_oddsDetailWin.checkedOddsType(1)']"))
        )
        # Click the button for Q2 handicap odds
        ah_odds_button.click()
        time.sleep(2)  # Optional: Wait for any potential data load

        # Grab all the text on the page for Q2 handicap odds
        handicap_odds_q2 = driver.page_source

        # Return a dictionary with all the extracted page sources
        odds_data = {
            "handicap_odds_q4": parse_htmlodds(handicap_odds_q4),
            "total_odds_q4": parse_htmlodds(total_odds_q4),
            "handicap_odds_q2": parse_htmlodds(handicap_odds_q2),
            "total_odds_q2": parse_htmlodds(total_odds_q2)
        }
        print(odds_data)
        return odds_data

    except Exception as e:
        print(f"An error occurred: {e}")
        return None


In [30]:
def open_nowgoal(driver, url):
    driver.get(url)
    wait = WebDriverWait(driver, 10)
    matchinfo = None
    pregame = None
    livegame = None
    
    try:
        matchinfo = grabmatchinfo(driver)
        pregame = grabpregame(driver)
        livegame= grablivegame(driver)
    except Exception as e:
        print(f"Failed to expand data or click trends button: {e}")
    
    nowgoaldict = {
        "matchinfo":matchinfo,
        "pregame":pregame,
        "livegame":livegame
    }
    return nowgoaldict

In [31]:
def parse_htmlodds(page_source):
    soup = BeautifulSoup(page_source, 'html.parser')
    text = soup.get_text(separator='\n', strip=True)
    
    
    return parse_raw_data(text)

In [32]:
def process_odds(nowgoaldict):
    # Iterate over each item in the 'livegame' dictionary and apply parse_odds
    parsed_livegame = {}
    for key, html_content in nowgoaldict["livegame"].items():
        # Apply parse_odds to each HTML string
        parsed_data = parse_odds(html_content)
        # Store the parsed data in the parsed_livegame dictionary
        parsed_livegame[key] = parsed_data

    # Update the nowgoaldict with the parsed live game data
    nowgoaldict["livegame"] = parsed_livegame
    
    
    
    return nowgoaldict

In [33]:
# Flattening the dictionary
def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = f'{parent_key}{sep}{k}' if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        elif isinstance(v, list):
            items.append((new_key, ', '.join(map(str, v))))  # Convert lists to comma-separated strings
        else:
            items.append((new_key, v))
    return dict(items)

In [34]:
def save_nowgoaldict(nowgoaldict):
        
   # Flatten matchinfo and pregame
    matchinfo_flat = flatten_dict(nowgoaldict["matchinfo"])
    pregame_flat = flatten_dict(nowgoaldict["pregame"])

    # Process livegame data, where each entry is a DataFrame
    livegame_df = pd.DataFrame()

    for key, df in nowgoaldict["livegame"].items():
        if isinstance(df, pd.DataFrame):
            df['Type'] = key  # Add a column for the type of odds
            # Add matchinfo and pregame data to each row
            for column, value in {**matchinfo_flat, **pregame_flat}.items():
                df[column] = value
            livegame_df = pd.concat([livegame_df, df], ignore_index=True)

    # Save the combined DataFrame to CSV
    combined_csv_path = 'nowgoalodds/combined_info.csv'
    livegame_df.to_csv(combined_csv_path, index=False)
    print(f"Combined info saved to {combined_csv_path}")

    # Debug: print the DataFrame for livegame to verify content
    print("Combined Live Game DataFrame:")
    print(livegame_df)

In [35]:
def get_combined_dataframe(nowgoaldict):
    # Flatten matchinfo and pregame
    matchinfo_flat = flatten_dict(nowgoaldict["matchinfo"])
    pregame_flat = flatten_dict(nowgoaldict["pregame"])

    # Process livegame data, where each entry is a DataFrame
    livegame_df = pd.DataFrame()

    for key, df in nowgoaldict["livegame"].items():
        if isinstance(df, pd.DataFrame):
            df['Type'] = key  # Add a column for the type of odds
            # Add matchinfo and pregame data to each row
            for column, value in {**matchinfo_flat, **pregame_flat}.items():
                df[column] = value
            livegame_df = pd.concat([livegame_df, df], ignore_index=True)

    return livegame_df

In [36]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import time
from pprint import pprint
from selenium.common.exceptions import NoSuchWindowException

logging_df = pd.DataFrame()

def save_logging_data(logging_df, timestamp, file_name):
    logging_df.to_csv(f"logs/{file_name}_{timestamp}.csv", index=False)


def main():
    global logging_df  # Declare logging_df as global to modify it within this function
    timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S')
    #537694
    driver = None
    try:
        driver = create_undetected_chrome_driver()
        for game_id in range(538028, 538257):  # Loop through game IDs
            url = f"https://live4.nowgoal15.com/oddscompbasket/{game_id}"
            nowgoaldict = open_nowgoal(driver, url)
            
            if nowgoaldict is None:
                print(f"Skipping game {game_id} due to missing or empty livegame data.")
                continue
            # Skip games where livegame data is missing or empty
            elif nowgoaldict.get("livegame") is None or nowgoaldict.get("matchinfo").get("sport") != 'National Basketball Association':
                print(f"Skipping game {game_id} due to missing or empty livegame data.")
                continue

            # Get combined DataFrame for the current game
            game_data = get_combined_dataframe(nowgoaldict)

            # Check if game_data is not empty
            if not game_data.empty:
                # Append to logging_df
                logging_df = pd.concat([logging_df, game_data], ignore_index=True)
            
            # Pretty print the contents of nowgoaldict for debugging
            print(f"Contents of nowgoaldict for game {game_id}:")
            pprint(nowgoaldict)

    except NoSuchWindowException:
        print("Browser window was closed. Continuing execution.")
        # Optionally handle cleanup or restart logic here
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        if driver:
            try:
                driver.quit()
            except Exception as e:
                print(f"Failed to quit the driver: {e}")

    # Save the logging data\
    save_logging_data(logging_df, timestamp, "regular_jan2024")
                     
if __name__ == "__main__":
    main()

Home Team: Washington Wizards, URL: //basketball.nowgoal15.com/team/summary/8, Image: https://basketball.nowgoal15.com/files/team/20180117153230.png?9
Away Team: Atlanta Hawks, URL: //basketball.nowgoal15.com/team/summary/13, Image: https://basketball.nowgoal15.com/files/team/20180117153007.png?9
Sport: National Basketball Association
Match Date: 12/31/2023 8:00:00 PM
Home Score: 126
Away Score: 130
Game State: Finished
Quarter Scores: (58-70,68-60)
Extracted Data: {'Bookmaker': 'Bet365', 'Status': ['Early', 'Live', 'Run'], 'Odds 1': ['0.91', '0.91', '0.74'], 'Handicap 1': ['-6.5', '-6.5', '-11.5'], 'Odds 2': ['0.91', '0.91', '1.05'], 'Odds 3': ['3.00', '3.00', None], 'Odds 4': ['1.41', '1.41', None], 'Odds 5': ['0.91', '0.91', '0.95'], 'Total': ['251.5', '252', '242.5'], 'Odds 6': ['0.91', '0.91', '0.80'], 'Trends': "_oddsDetailWin.open('538028', '8', 'Bet365', 0, (_halfTime ? '3' : '6'))"}
  Bookmaker              Status              Odds 1           Handicap 1  \
0    Bet365  [Early